# Toolkits

Toolkits são conjuntos de ferramentas. Para alguns toolkits, agent específicos já foram criado na estrutura do LangChain e é isto que vamos explorar agora.

https://python.langchain.com/v0.1/docs/integrations/toolkits/

## Pandas Dataframe

In [1]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import pandas as pd
_= load_dotenv(find_dotenv())

chat = ChatOpenAI(model='gpt-4o-mini')
df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.shape

(891, 12)

In [3]:
agent = create_pandas_dataframe_agent(
    chat,
    df,
    verbose=True,
    agent_type='tool-calling',
    allow_dangerous_code=True
)

In [4]:
agent.invoke({'input': 'Quantas linhas tem na tabela?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


891A tabela possui 891 linhas.

> Finished chain.


{'input': 'Quantas linhas tem na tabela?',
 'output': 'A tabela possui 891 linhas.'}

In [6]:
agent.invoke({'input': 'Qual a média de idade dos passageiros?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Age'].mean()"}`


29.69911764705882A média de idade dos passageiros é aproximadamente 29.7 anos.

> Finished chain.


{'input': 'Qual a média de idade dos passageiros?',
 'output': 'A média de idade dos passageiros é aproximadamente 29.7 anos.'}

In [7]:
agent.invoke({'input': 'Quantos passageiros sobreviveram?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Survived'].sum()"}`


342No total, 342 passageiros sobreviveram.

> Finished chain.


{'input': 'Quantos passageiros sobreviveram?',
 'output': 'No total, 342 passageiros sobreviveram.'}

## SQL Database

In [8]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri('sqlite:///arquivos/Chinook.db')

In [9]:
from langchain_community.agent_toolkits.sql.base import create_sql_agent

agent_executor = create_sql_agent(
    chat,
    db=db,
    agent_type='tool-calling',
    verbose=True
)

In [10]:
for tool in agent_executor.tools:
    print(tool.name)
    print(tool.description)
    print()

sql_db_query
Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema
Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables
Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker
Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [11]:
agent_executor.invoke({'input': 'Me descreva a base de dados'})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Album'}`



CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/
Invoking: `sql_db_schema` with `{'table_names': 'Artist'}`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Invoking: `sql_db_schema` with `{'table_names': 'Customer'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT 

{'input': 'Me descreva a base de dados',
 'output': 'A base de dados contém as seguintes tabelas, cada uma com suas respectivas colunas:\n\n1. **Album**\n   - AlbumId (INTEGER)\n   - Title (NVARCHAR(160))\n   - ArtistId (INTEGER) - Chave estrangeira referindo-se à tabela Artist\n\n2. **Artist**\n   - ArtistId (INTEGER)\n   - Name (NVARCHAR(120))\n\n3. **Customer**\n   - CustomerId (INTEGER)\n   - FirstName (NVARCHAR(40))\n   - LastName (NVARCHAR(20))\n   - Company (NVARCHAR(80))\n   - Address (NVARCHAR(70))\n   - City (NVARCHAR(40))\n   - State (NVARCHAR(40))\n   - Country (NVARCHAR(40))\n   - PostalCode (NVARCHAR(10))\n   - Phone (NVARCHAR(24))\n   - Fax (NVARCHAR(24))\n   - Email (NVARCHAR(60))\n   - SupportRepId (INTEGER) - Chave estrangeira referindo-se à tabela Employee\n\n4. **Employee**\n   - EmployeeId (INTEGER)\n   - LastName (NVARCHAR(20))\n   - FirstName (NVARCHAR(20))\n   - Title (NVARCHAR(30))\n   - ReportsTo (INTEGER) - Chave estrangeira referindo-se à tabela Employee\n  

In [12]:
agent_executor.invoke({'input': 'Qual artista possui mais albuns?'})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Artist'}`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Invoking: `sql_db_schema` with `{'table_names': 'Album'}`



CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT Artist.Name, COUNT(Album.AlbumId) AS AlbumCount FROM Artist INNER JOIN Album ON Artist.ArtistId 

{'input': 'Qual artista possui mais albuns?',
 'output': 'O artista com mais álbuns é **Iron Maiden**, com um total de 21 álbuns.'}

In [13]:
agent_executor.invoke({'input': 'Qual o cliente que mais comprou?'})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José dos Camp

{'input': 'Qual o cliente que mais comprou?',
 'output': 'Os clientes que mais compraram, com 7 compras cada, são:\n\n1. Luís Gonçalves\n2. Leonie Köhler\n3. François Tremblay\n4. Bjørn Hansen\n5. František Wichterlová\n6. Helena Holý\n7. Astrid Gruber\n8. Daan Peeters\n9. Kara Nielsen\n10. Eduardo Martins\n\nTodos esses clientes têm um total de 7 compras.'}